In [1]:
import numpy as np
from scipy.sparse import lil_matrix, save_npz, load_npz
from multiprocessing import Pool, Manager, Value
from tqdm import tqdm

import os
import sys
sys.path.append('../..')
sys.path.append('../../RecStudio')
import pickle
import numpy as np
from data.MIND.process import MINDSeqDataset
from dataset import CTRDataset
from typing import Dict
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer
from sentence_transformers.util import cos_sim
from collections import defaultdict
import math

BATCH_SIZE = 128
MAX_LEN = 512
MODEL_ID = '../../mxbai-embed-large-v1'

/data1/home/xingmei/.conda/envs/rec/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
category = 'Bili_2M'

In [ ]:
# ----------------------------Get cosine similarity of co-occurence matrix -------------------------

if not 'guide_model_text_embedding' in locals().keys():
    guide_model_text_embedding = torch.load(f'guide_model_text_embeddings/{category}.pt')
if not 'co_occurrence' in locals().keys():
    co_occurrence = load_npz(f'co_occurrence/{category}.npz')
    co_occurrence = co_occurrence.tocoo()

sim = []
for k in tqdm(range(len(co_occurrence.data))):
    count = co_occurrence.data[i]
    item_i = torch.tensor(co_occurrence.row[k], dtype=torch.int32)
    item_j = torch.tensor(co_occurrence.col[k], dtype=torch.int32)
    item_i_text_emb = guide_model_text_embedding(item_i)
    item_j_text_emb = guide_model_text_embedding(item_j)
    sim.append(cos_sim(item_i_text_emb, item_j_text_emb)[0][0].item())

with open(f'cos_sim_of_co_occurence/{category}.pkl', 'wb') as f:
    pickle.dump(sim, f)

In [3]:
# ----------------------------------------Filter pairs-----------------------------------------------
if not 'co_occurrence' in locals().keys():
    co_occurrence = load_npz(f'co_occurrence/{category}.npz')
    co_occurrence = co_occurrence.tocoo()
if not 'sim' in locals().keys():
    sim = pickle.load(open(f'cos_sim_of_co_occurence/{category}.pkl', 'rb'))

In [6]:
len(sim)

334673404

In [7]:
len(co_occurrence.data)

334673404

In [8]:
len([_ for _ in co_occurrence.data if _ > 0.1])

175990

In [9]:
len([_ for _ in co_occurrence.data if _ > 0.3])

6836

In [10]:
len([_ for _ in co_occurrence.data if _ > 0.5])

756

In [11]:
len([_ for _ in sim if _ > 0.65])

866920

In [12]:
len([_ for _ in sim if _ > 0.8])

26698

In [13]:
len([_ for _ in sim if _ > 0.9])

4994

In [14]:
CO_OCCURENCE_THRESHOLD = 0.2
pairs = set()
for i in tqdm(range(len(sim))):
    if co_occurrence.data[i] > CO_OCCURENCE_THRESHOLD and \
        (co_occurrence.col[i], co_occurrence.row[i]) not in pairs and \
        (co_occurrence.row[i] != co_occurrence.col[i]):
        pairs.add((co_occurrence.row[i], co_occurrence.col[i]))
print(f"Length of pairs: {len(pairs)}")

  0%|          | 248869/334673404 [00:00<02:14, 2488502.52it/s]

100%|██████████| 334673404/334673404 [01:51<00:00, 2995549.48it/s]

Length of pairs: 12700


In [4]:
CO_OCCURENCE_THRESHOLD = 0.2
COS_SIM_THRESHOLD = 0.7
pairs_27 = set()
for i in tqdm(range(len(sim))):
    if co_occurrence.data[i] > CO_OCCURENCE_THRESHOLD and sim[i] > COS_SIM_THRESHOLD and \
        (co_occurrence.col[i], co_occurrence.row[i]) not in pairs_27 and \
        (co_occurrence.row[i] != co_occurrence.col[i]):
        pairs_27.add((co_occurrence.row[i], co_occurrence.col[i]))
print(f"Length of pairs: {len(pairs_27)}")

  0%|          | 0/334673404 [00:00<?, ?it/s]

100%|██████████| 334673404/334673404 [01:46<00:00, 3139443.22it/s]

Length of pairs: 1305


In [16]:
CO_OCCURENCE_THRESHOLD = 0.2
COS_SIM_THRESHOLD = 0.65
pairs_265 = set()
for i in tqdm(range(len(sim))):
    if co_occurrence.data[i] > CO_OCCURENCE_THRESHOLD and sim[i] > COS_SIM_THRESHOLD and \
        (co_occurrence.col[i], co_occurrence.row[i]) not in pairs_265 and \
        (co_occurrence.row[i] != co_occurrence.col[i]):
        pairs_265.add((co_occurrence.row[i], co_occurrence.col[i]))
print(f"Length of pairs: {len(pairs_265)}")

  0%|          | 555454/334673404 [00:00<01:58, 2827629.59it/s]

100%|██████████| 334673404/334673404 [01:41<00:00, 3300217.35it/s]

Length of pairs: 2051


In [17]:
CO_OCCURENCE_THRESHOLD = 0.15
COS_SIM_THRESHOLD = 0.65
pairs_1565 = set()
for i in tqdm(range(len(sim))):
    if co_occurrence.data[i] > CO_OCCURENCE_THRESHOLD and sim[i] > COS_SIM_THRESHOLD and \
        (co_occurrence.col[i], co_occurrence.row[i]) not in pairs_1565 and \
        (co_occurrence.row[i] != co_occurrence.col[i]):
        pairs_1565.add((co_occurrence.row[i], co_occurrence.col[i]))
print(f"Length of pairs: {len(pairs_1565)}")

100%|██████████| 334673404/334673404 [01:41<00:00, 3295238.31it/s]

Length of pairs: 4427


In [18]:
CO_OCCURENCE_THRESHOLD = 0.1
COS_SIM_THRESHOLD = 0.65
pairs_165 = set()
for i in tqdm(range(len(sim))):
    if co_occurrence.data[i] > CO_OCCURENCE_THRESHOLD and sim[i] > COS_SIM_THRESHOLD and \
        (co_occurrence.col[i], co_occurrence.row[i]) not in pairs_165 and \
        (co_occurrence.row[i] != co_occurrence.col[i]):
        pairs_165.add((co_occurrence.row[i], co_occurrence.col[i]))
print(f"Length of pairs: {len(pairs_165)}")

100%|██████████| 334673404/334673404 [01:41<00:00, 3303694.85it/s]

Length of pairs: 10989


In [5]:
CO_OCCURENCE_THRESHOLD = 0.1
COS_SIM_THRESHOLD = 0.7
pairs_17 = set()
for i in tqdm(range(len(sim))):
    if co_occurrence.data[i] > CO_OCCURENCE_THRESHOLD and sim[i] > COS_SIM_THRESHOLD and \
        (co_occurrence.col[i], co_occurrence.row[i]) not in pairs_17 and \
        (co_occurrence.row[i] != co_occurrence.col[i]):
        pairs_17.add((co_occurrence.row[i], co_occurrence.col[i]))
print(f"Length of pairs: {len(pairs_17)}")

  0%|          | 152915/334673404 [00:00<03:38, 1528994.71it/s]

100%|██████████| 334673404/334673404 [01:39<00:00, 3374932.42it/s]

Length of pairs: 5907


In [6]:
CO_OCCURENCE_THRESHOLD = 0.1
COS_SIM_THRESHOLD = 0.8
pairs_18 = set()
for i in tqdm(range(len(sim))):
    if co_occurrence.data[i] > CO_OCCURENCE_THRESHOLD and sim[i] > COS_SIM_THRESHOLD and \
        (co_occurrence.col[i], co_occurrence.row[i]) not in pairs_18 and \
        (co_occurrence.row[i] != co_occurrence.col[i]):
        pairs_18.add((co_occurrence.row[i], co_occurrence.col[i]))
print(f"Length of pairs: {len(pairs_18)}")

100%|██████████| 334673404/334673404 [01:40<00:00, 3335099.54it/s]

Length of pairs: 1662


In [20]:
with open(f'pairs/{category}.pkl', 'wb') as f:
    pickle.dump(pairs_165, f)

In [ ]:
with open(f'pairs/{category}.pkl', 'wb') as f:
    pickle.dump(pairs_27, f)

In [ ]:
with open(f'pairs/{category}_1_7.pkl', 'wb') as f:
    pickle.dump(pairs_17, f)

In [ ]:
with open(f'pairs/{category}_1_8.pkl', 'wb') as f:
    pickle.dump(pairs_18, f)

# Stats pairs distribution

In [3]:
with open(f'pairs/{category}.pkl', 'rb') as f:
    pairs_165 = pickle.load(f)

In [4]:
from collections import defaultdict
item_cnt = defaultdict(int)
for i, j in pairs_165:
    item_cnt[i] += 1
    item_cnt[j] += 1
len(item_cnt.keys())

1571

In [5]:
all_items = torch.tensor(list(item_cnt.keys()))

if not 'guide_model_text_embedding' in locals().keys():
    guide_model_text_embedding = torch.load(f'guide_model_text_embeddings/{category}.pt')
guide_model_text_embedding = guide_model_text_embedding.cuda()

sim = 0
cnt = 0
for i in tqdm(range(len(all_items))):
    item_i = all_items[i].unsqueeze(-1).cuda()
    emb_i = guide_model_text_embedding(item_i)
    emb_i = torch.nn.functional.normalize(emb_i, p=2, dim=1)
    item_j = all_items[i + 1:]
    for start in range(0, len(item_j), BATCH_SIZE):
        end = min(start + BATCH_SIZE, len(all_items))
        batch_item_j = item_j[start:end].cuda()
        emb_j = guide_model_text_embedding(batch_item_j)
        emb_j = torch.nn.functional.normalize(emb_j, p=2, dim=1)
        sim += (emb_i * emb_j).sum()
        cnt += end - start

avg_sim = sim / cnt
avg_sim

100%|██████████| 1571/1571 [00:01<00:00, 978.54it/s] 


tensor(0.4150, device='cuda:0')